# Assessment for Advanced Data Science

## Christian Cabrera, Carl Henrik Ek and Neil D. Lawrence

### 29th October 2021


## Introduction

Welcome to the course assessment for the Advanced Data Science unit. In this assessment you will build a prediction system for UK house prices.

Your prediction system will be based on data from the UK Price Paid data available [here](https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads). By combining this data with the UK Office for National Statistics data on the latitude/longitude of postcodes (available [here](https://www.getthedata.com/open-postcode-geo)) you will have a record of house prices and their approximate latitude/longitude. Due to the size of these data you will use a relational database to handle them.

To make predictions of the house price you will augment your data with information obtained from Open Street Map: an open license source of mapping information. You will use the techniques you have learnt in the course to indentify and incorporate useful features for house price prediction.

Alongside your implementation you will provide a short repository overview describing how you have implemented the different parts of the project and where you have placed those parts in your code repository. You will submit your code alongside a version of this notebook that will allow your examiner to understand and reconstruct the thinking behind your analysis. This notebook is structured to help you in creating that description and allow you to understand how we will allocate the marks. You should make use of the Fynesse framework (<https://github.com/lawrennd/fynesse_template>) for structuring your code.

Remember the notebook you create should _tell a story_, any code that is not critical to that story can safely be placed into the associated analysis library and imported for use (structured as given in the Fynesse template)

The maximum total mark for this assessment is 20. That mark is split into Three Questions below, each worth 5 marks each. Then a final 5 marks will be given for the quality, structure and reusability of the code and analysis you produce giving 20 marks in total.


### Useful Links

You may find some of the following links useful when building your system.

University instuctions on Security and Privacy with AWS.

https://help.uis.cam.ac.uk/service/network-services/hosting-services/AWS/aws-security-privacy

Security Rules in AWS

https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/USER_VPC.Scenarios.html#USER_VPC.Scenario4


### Installing Your Library

One artefact to be included in your submission is a python library structured according to the "Access, Assess, Address" standard for data science solutions. You will submit this library alongside your code. Use the cell below to perform the necessary installation instructions for your library.

You should base your module on the template repository given by the Fynesse template repository. That should make it `pip` installable as below.


In [ ]:
# Install your library here, if running on Colab
%pip install git+https://github.com/andylolu2/ads-assignment.git

Your own library should be installed in the line above, then you can import it as usual (where you can either replace `fynesse` with the name you've given your analysis module or you can leave the name as `fynesse` as you prefer).


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fynesse import access, assess, address, config, utils

## Question 1. Accessing a Database of House Prices, Latitudes and Longitudes

The UK price paid data for housing in dates back to 1995 and contains millions of transactions. The size of the data makes it unwieldy to manipulate directly in python frameworks such as `pandas`. As a result we will host the data in a _relational database_.

Using the following ideas.

1. A cloud hosted database (such as MariaDB hosted on the AWS RDS service).
2. The SQL language wrapped in appropriately structured python code.
3. Joining of two databases.

You will construct a database containing tables that contain all house prices, latitudes and longitudes from the UK house price data base since 1995.

You will likely find the following resources helpful.

1. Lecture 1, 2 and 3.
2. Lab class 1 and 2.
3. The UK Price Paid data for houses: <https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads>
4. The UK ONS Data base of postcode latitude and longitudes: <https://www.getthedata.com/open-postcode-geo>

Below we provide codeboxes and hints to help you develop your answer.

_The main knowledge you need to do a first pass through this question will have been taught by the end of Lab Session 2 (11th November 2021). You will likely want to review your answer as part of **refactoring** your code and analysis pipeline shortly before hand in._

_5 Marks_


### Task A

Set up the database. You'll need to set up a database on AWS. You were guided in how to do this in the lab sessions. You should be able to use the same database instance you created in the lab, or you can delete that and start with a fresh instance. You'll remember from the lab that the database requires credentials (username, password) to access. It's good practice to store those credentials _outside_ the notebook so you don't accidentally share them by e.g. checking code into a repository.

Call the database you use for this assessment `property_prices`.

In [ ]:
# Set up credentials by adding "db_username", "db_password", "db_host", "db_port" to _config.yml.
# Otherwise, set the values here by calling `set_credentials`.


def set_credentials(username, password, host, port):
    conf = config.config
    conf["db_username"] = username
    conf["db_password"] = password
    conf["db_host"] = host
    conf["db_port"] = port


for k in ["db_username", "db_password", "db_host", "db_port"]:
    assert k in config.config


In [ ]:
# Create database
with access.create_connection() as conn:
    with conn.cursor() as cursor:
        cursor.execute("SET SQL_MODE = `NO_AUTO_VALUE_ON_ZERO`")
        cursor.execute("SET time_zone = `+00:00`")
        cursor.execute(
            "CREATE DATABASE IF NOT EXISTS `property_prices` DEFAULT CHARACTER SET utf8 COLLATE utf8_bin"
        )
    conn.commit()


In [ ]:
# Confirm database exists
access.sql_read("SHOW DATABASES")

### Task B

Create a database table called `pp_data` containing all the UK Price Paid data from the [gov.uk site](https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads). You'll need to prepare a new table to receive the data and upload the UK Price Paid data to your database instance. The total data is over 3 gigabytes in size. We suggest that rather than downloading the full data in CSV format, you use the fact that they have split the data into years and into different parts per year. For example, the first part of the data for 2018 is stored at <http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018-part1.csv>. Each of these files is less than 100MB and can be downloaded very quickly to local disk, then uploaded using

```
LOCAL DATA LOAD INFILE 'filename' INTO TABLE `table_name`
FIELDS TERMINATED BY ','
LINES STARTING BY '' TERMINATED BY '\n';
```

_Note_ this command should be wrapped and placed in an appropriately structured python module.

Each 'data part' should be downloadable from the `gov.uk` site and uploadable to your database instance in a couple of seconds. By looping across the years and different parts, you should be able to robustly upload this large data set to your database instance in a matter of minutes.

You may find the following schema useful in creation of your database:

```
--
-- Table structure for table `pp_data`
--
DROP TABLE IF EXISTS `pp_data`;
CREATE TABLE IF NOT EXISTS `pp_data` (
  `transaction_unique_identifier` tinytext COLLATE utf8_bin NOT NULL,
  `price` int(10) unsigned NOT NULL,
  `date_of_transfer` date NOT NULL,
  `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
  `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
  `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
  `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
  `primary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
  `secondary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
  `street` tinytext COLLATE utf8_bin NOT NULL,
  `locality` tinytext COLLATE utf8_bin NOT NULL,
  `town_city` tinytext COLLATE utf8_bin NOT NULL,
  `district` tinytext COLLATE utf8_bin NOT NULL,
  `county` tinytext COLLATE utf8_bin NOT NULL,
  `ppd_category_type` varchar(2) COLLATE utf8_bin NOT NULL,
  `record_status` varchar(2) COLLATE utf8_bin NOT NULL,
  `db_id` bigint(20) unsigned NOT NULL
) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1 ;
```

This schema is written by Dale Potter and can be found on Github here: <https://github.com/dalepotter/uk_property_price_data/blob/master/create_db.sql>

You may also find it helpful to set up the following indexes in the database

```
--
-- Indexes for table `pp_data`
--
ALTER TABLE `pp_data`
ADD PRIMARY KEY (`db_id`);
MODIFY `db_id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,AUTO_INCREMENT=1;
CREATE INDEX `pp.postcode` USING HASH
  ON `pp_data`
    (postcode);
CREATE INDEX `pp.date` USING HASH
  ON `pp_data`
    (date_of_transfer);
```


In the box below, briefly describe what the schema is doing and why we will find it useful to create the indexes we have for the table we've created.


#### Answer

The schema is unpacking the columns in the property prices csv data into suitable data types. 

The indexes are useful for speeding up lookup / joins on particular columns.

In [ ]:
# Create pp_data schema
with access.create_connection("property_prices") as conn:
    with conn.cursor() as cursor:
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS `pp_data` (
                `transaction_unique_identifier` tinytext COLLATE utf8_bin NOT NULL,
                `price` int(10) unsigned NOT NULL,
                `date_of_transfer` date NOT NULL,
                `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
                `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
                `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
                `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
                `primary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
                `secondary_addressable_object_name` tinytext COLLATE utf8_bin NOT NULL,
                `street` tinytext COLLATE utf8_bin NOT NULL,
                `locality` tinytext COLLATE utf8_bin NOT NULL,
                `town_city` tinytext COLLATE utf8_bin NOT NULL,
                `district` tinytext COLLATE utf8_bin NOT NULL,
                `county` tinytext COLLATE utf8_bin NOT NULL,
                `ppd_category_type` varchar(2) COLLATE utf8_bin NOT NULL,
                `record_status` varchar(2) COLLATE utf8_bin NOT NULL,
                `db_id` bigint(20) unsigned NOT NULL
                ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1
        """
        )
        cursor.execute("ALTER TABLE `pp_data` ADD PRIMARY KEY (`db_id`)")
        cursor.execute(
            "ALTER TABLE `pp_data` MODIFY `db_id` bigint(20) unsigned NOT NULL AUTO_INCREMENT, AUTO_INCREMENT=1"
        )
    conn.commit()


In [ ]:
from pathlib import Path

# Download and upload all data

save_dir = Path("data")
part_url_format = config.config["pp_data_part_url_format"]
url_format = config.config["pp_data_url_format"]

urls = [url_format.format(year=2022)]
paths = [save_dir / "2022.csv"]
for year in range(1995, 2022):
    for part in [1, 2]:
        urls.append(part_url_format.format(year=year, part=part))
        paths.append(save_dir / f"{year}-{part}.csv")

# access.reset(table="pp_data", database="property_prices")

for url, path in zip(urls, paths):
    access.download(url, path)
    access.upload(path, table="pp_data", database="property_prices", delimiter='"')


In [ ]:
# Add index after uploading
with access.create_connection("property_prices") as conn:
    with conn.cursor() as cursor:
        cursor.execute("CREATE INDEX `pp.postcode` USING HASH ON `pp_data` (postcode)")
        cursor.execute(
            "CREATE INDEX `pp.date` USING HASH ON `pp_data` (date_of_transfer)"
        )
    conn.commit()

### Task C

Create a database table called `postcode_data` containing the ONS Postcode information. <GetTheData.com> has organised data derived from the UK Office for National Statistics into a convenient CSV file. You can find details [here](https://www.getthedata.com/open-postcode-geo).

The data you need can be found at this url: <https://www.getthedata.com/downloads/open_postcode_geo.csv.zip>. It will need to be unzipped before use.

You may find the following schema useful for the postcode data (developed by Christian and Neil)

```
USE `property_prices`;
--
-- Table structure for table `postcode_data`
--
DROP TABLE IF EXISTS `postcode_data`;
CREATE TABLE IF NOT EXISTS `postcode_data` (
  `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
  `status` enum('live','terminated') NOT NULL,
  `usertype` enum('small', 'large') NOT NULL,
  `easting` int unsigned,
  `northing` int unsigned,
  `positional_quality_indicator` int NOT NULL,
  `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
  `lattitude` decimal(11,8) NOT NULL,
  `longitude` decimal(10,8) NOT NULL,
  `postcode_no_space` tinytext COLLATE utf8_bin NOT NULL,
  `postcode_fixed_width_seven` varchar(7) COLLATE utf8_bin NOT NULL,
  `postcode_fixed_width_eight` varchar(8) COLLATE utf8_bin NOT NULL,
  `postcode_area` varchar(2) COLLATE utf8_bin NOT NULL,
  `postcode_district` varchar(4) COLLATE utf8_bin NOT NULL,
  `postcode_sector` varchar(6) COLLATE utf8_bin NOT NULL,
  `outcode` varchar(4) COLLATE utf8_bin NOT NULL,
  `incode` varchar(3)  COLLATE utf8_bin NOT NULL,
  `db_id` bigint(20) unsigned NOT NULL
) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
```

And again you'll want to set up indices for your table.

```
ALTER TABLE `postcode_data`
ADD PRIMARY KEY (`db_id`);
MODIFY `db_id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,AUTO_INCREMENT=1;
CREATE INDEX `po.postcode` USING HASH
  ON `postcode_data`
    (postcode);
```

And you can load the CSV file into the table in one "INFILE".

```
LOAD DATA LOCAL INFILE 'open_postcode_geo.csv' INTO TABLE `postcode_data`
FIELDS TERMINATED BY ','
LINES STARTING BY '' TERMINATED BY '\n';
```


In [ ]:
# Create postcode_data schema
with access.create_connection("property_prices") as conn:
    with conn.cursor() as cursor:
        cursor.execute("DROP TABLE IF EXISTS `postcode_data`")
        cursor.execute(
            """
                CREATE TABLE IF NOT EXISTS `postcode_data` (
                `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
                `status` enum('live','terminated') NOT NULL,
                `usertype` enum('small', 'large') NOT NULL,
                `easting` int unsigned,
                `northing` int unsigned,
                `positional_quality_indicator` int NOT NULL,
                `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
                `latitude` decimal(11,8) NOT NULL,
                `longitude` decimal(10,8) NOT NULL,
                `postcode_no_space` tinytext COLLATE utf8_bin NOT NULL,
                `postcode_fixed_width_seven` varchar(7) COLLATE utf8_bin NOT NULL,
                `postcode_fixed_width_eight` varchar(8) COLLATE utf8_bin NOT NULL,
                `postcode_area` varchar(2) COLLATE utf8_bin NOT NULL,
                `postcode_district` varchar(4) COLLATE utf8_bin NOT NULL,
                `postcode_sector` varchar(6) COLLATE utf8_bin NOT NULL,
                `outcode` varchar(4) COLLATE utf8_bin NOT NULL,
                `incode` varchar(3)  COLLATE utf8_bin NOT NULL,
                `db_id` bigint(20) unsigned NOT NULL
                ) DEFAULT CHARSET=utf8 COLLATE=utf8_bin;
        """
        )
        cursor.execute("ALTER TABLE `postcode_data` ADD PRIMARY KEY (`db_id`)")
        cursor.execute(
            "ALTER TABLE `postcode_data` MODIFY `db_id` bigint(20) unsigned NOT NULL AUTO_INCREMENT,AUTO_INCREMENT=1"
        )
    conn.commit()

In [ ]:
!mkdir -p ./data/postcode
!wget -P ./data/postcode https://www.getthedata.com/downloads/open_postcode_geo.csv.zip
!unzip ./data/postcode/open_postcode_geo.csv.zip -d ./data/postcode

In [ ]:
from pathlib import Path

access.upload(
    Path("data") / "postcode" / "open_postcode_geo.csv", 
    table="postcode_data", 
    database="property_prices"
)

In [ ]:
with access.create_connection("property_prices") as conn:
    with conn.cursor() as cursor:
        cursor.execute("CREATE INDEX `po.postcode` USING HASH ON `postcode_data` (postcode)")
    conn.commit()

In [ ]:
access.sql_read("SELECT * FROM postcode_data LIMIT 5", "property_prices")

### Task D

These data can now be joined to form a new table that contains house price paid and latitude longitude of the house. We could create a new table that contains all this information. However, the computation of that table will take some time because of the size of the two existing tables in the join.

Instead, we're going to exploit the nature of the task. To build our prediction model, we're going to use the prices for a particular region in a given time period. This means we can select that region and time period and build the joined data only from the relevent rows from the two tables. This will save time on the join.

Whether this is a good idea or not in a live system will depend on how often these predictions are required. If it's very often, it would likely be better to store the entired database joined, because the one-off cost for that join is amortised across all the future predictions. If only a few predictions are required (like in our lab class) then doing that join on the fly might be better. In that case you can make use of an _inner join_ for this data set creation.

```
USE `property_prices`;
--
-- Table structure for table `prices_coordinates_data`
--
DROP TABLE IF EXISTS `prices_coordinates_data`;
CREATE TABLE IF NOT EXISTS `prices_coordinates_data` (
  `price` int(10) unsigned NOT NULL,
  `date_of_transfer` date NOT NULL,
  `postcode` varchar(8) COLLATE utf8_bin NOT NULL,
  `property_type` varchar(1) COLLATE utf8_bin NOT NULL,
  `new_build_flag` varchar(1) COLLATE utf8_bin NOT NULL,
  `tenure_type` varchar(1) COLLATE utf8_bin NOT NULL,
  `locality` tinytext COLLATE utf8_bin NOT NULL,
  `town_city` tinytext COLLATE utf8_bin NOT NULL,
  `district` tinytext COLLATE utf8_bin NOT NULL,
  `county` tinytext COLLATE utf8_bin NOT NULL,
  `country` enum('England', 'Wales', 'Scotland', 'Northern Ireland', 'Channel Islands', 'Isle of Man') NOT NULL,
  `lattitude` decimal(11,8) NOT NULL,
  `longitude` decimal(10,8) NOT NULL,
  `db_id` bigint(20) unsigned NOT NULL
) DEFAULT CHARSET=utf8 COLLATE=utf8_bin AUTO_INCREMENT=1 ;


```


In [ ]:
lat, long = config.config["sample_coords"]["christs"]
bbox = utils.bbox(lat, long, 500, 500)

assess.query_price(
    date_min="2018-02-29", 
    date_max="2019-02-29", 
    bbox=bbox, 
    limit=5,
)


## Question 2. Accessing OpenStreetMap and Assessing the Available Features

In question 3 you will be given the task of constructing a prediction system for property price levels at a given location. We expect that knowledge of the local region around the property should be helpful in making those price predictions. To evaluate this we will now look at [OpenStreetMap](https://www.openstreetmap.org) as a data source.

The tasks below will guide you in accessing and assessing the OpenStreetMap data. The code you write will eventually be assimilated in your python module, but documentation of what you've included and why should remain in the notebook below.

Accessing OpenStreetMap through its API can be done using the python library `osmx`. Using what you have learned about the `osmx` interface in the lectures, write general code for downloading points of interest and other relevant information that you believe may be useful for predicting house prices. Remembering the perspectives we've taken on _data science as debugging_, the remarks we've made when discussing _the data crisis_ of the importance of reusability in data analysis, and the techniques we've explored in the labsessions for visualising features and exploring their correlation use the notebook to document your assessment of the OpenStreetMap data as a potential source of data.

The knowledge you need to do a first pass through this question will have been taught by end of lab session three (16th November 2021). You will likely want to review your answer as part of _refactoring_ your code and analysis pipeline shortly before hand in.

You should write reusable code that allows you to explore the characteristics of different points of interest. Looking ahead to question 3 you'll want to incorporate these points of interest in your prediction code.

_5 marks_


In [ ]:
import math
import warnings

import osmnx
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import seaborn as sns

dist = 500  # in meters

lat, long = config.config["sample_coords"]["christs"]
bbox1 = utils.bbox(lat, long, dist, dist)

lat, long = config.config["sample_coords"]["hammersmith"]
bbox2 = utils.bbox(lat, long, dist, dist)

#### Access OpenStreetMap

In [ ]:
def pois_from_bbox(bbox, tags):
    gdf = osmnx.geometries_from_bbox(*bbox, tags=tags)
    gdf = gdf.to_crs(crs=3857)
    gdf["geometry_area"] = gdf.area
    gdf = gdf.to_crs(crs=4326)
    return gdf

##### Tags

- Some manual trial and error is done test out different tags to download from OpenStreetMap. 
- The more detailed description about each tag (90K+!) in OpenStreetMap can be found on https://wiki.openstreetmap.org/wiki/Map_features.
- It was not possible to explore every tag but the most popular housing-related tags has been included.
- It is possible that there are some more useful tag out there which could be insightful. 

In [ ]:
tags = {
    "building": True, 
    "residential": True, 
    "shop": True, 
    "natural": True, 
    "amenity": True, 
    "leisure": True
}

In [ ]:
# Example
pois_from_bbox(bbox1, tags).head(2)

#### Assess features from OpenStreetMap

#### Data types and dealing with missing features

Q: What are the data types of each column? How to appropriately fill in the missing features?

In [ ]:
def clean_data(*bboxs):
    # built-in conversion seems to work well, and using <NA> to handle missing values seems reasonable
    gdf = pd.concat([pois_from_bbox(bbox, tags) for bbox in bboxs])
    return gdf.convert_dtypes()

gdf = clean_data(bbox1, bbox2)

print("Data types:")

dtypes = gdf.dtypes.astype(str).to_frame("dtype").reset_index()
print(dtypes.groupby("dtype")["index"].agg(list))

#### Sparsity of features

Q: How much of the OpenStreetMap data is actually just null values? What features carry the least number of null values?

In [ ]:
%%capture --no-display

def sparsity():
    gdf = clean_data(bbox1, bbox2)
    feature_sparsity = gdf.notna().mean().sort_values(ascending=False)
    return feature_sparsity

feature_sparsity = sparsity()
fig = plt.figure(figsize=(10, 4))

ax = fig.add_subplot(1, 3, (1, 2))
ax = feature_sparsity.iloc[:25].plot(kind="bar", width=0.9, color="royalblue", ax=ax)
ax.set_xlabel("Top 25 OSM features")
ax.set_ylabel("Proportion of non-null entries")

ax = fig.add_subplot(1, 3, 3)
ax = feature_sparsity.plot(kind="hist", bins=20, color="royalblue", edgecolor="w")
ax.set_yscale("log")
ax.set_xlabel("Proportion of non-null entries")
ax.set_ylabel("Number of features")

fig.tight_layout()

Key observations
- Most features do not occur in more than 5% of the rows. (They are usually nulls).
- Less than 20 features contain more than 20% of non-null rows. 

#### Cardinality of the features

Q: What is the cardinality of each feature? Is a feature heavily unique or categorical?

In [ ]:
%%capture --no-display
def top_features(n: int):
    gdf = clean_data(bbox1, bbox2)
    gdf = gdf.drop(columns=["nodes", "ways"])

    num_unique = gdf.nunique().to_frame("num_unique")
    num_not_na = gdf.notna().sum().to_frame("num_not_na")
    dtype = gdf.dtypes.to_frame("dtype")
    dtype["dtype"][gdf.applymap(type).eq(str).any()] = pd.StringDtype()

    features = pd.merge(num_unique, num_not_na, right_index=True, left_index=True)
    features = pd.merge(features, dtype, right_index=True, left_index=True)
    features["uniqueness"] = features["num_unique"] / features["num_not_na"]
    features = features.sort_values("num_not_na", ascending=False)

    return features[:n]

fig, [ax1, ax2, ax3] = plt.subplots(3, 1, sharex=True, figsize=(6, 6))

n = 25
top_n = top_features(n)
top_n["uniqueness"].plot(kind="bar", width=0.9, color="royalblue", colormap="viridis", ax=ax1)
ax1.set_ylabel("Prop. unique entries")

top_n["num_unique"].plot(kind="bar", width=0.9, color="royalblue", ax=ax2)
ax2.set_ylabel("No. unique entries")
ax2.set_yscale("log")

top_n["num_not_na"].plot(kind="bar", width=0.9, color="royalblue", ax=ax3)
ax3.set_xlabel(f"Top {n} OSM features")
ax3.set_ylabel("No. non-na entries")

fig.tight_layout()

Key observations / important notes:
- Roughly half of the top 20 OSM features are not categorical. (e.g. `name`, `brand`, `website`)
- This is **not** representative of the whole OSM dataset. As we are only analysing a subset of the OSM dataset, 
    some non-categorical features might appear as categorical (e.g. `addr: city`)

#### Feature occurrence correlation

Q: What features are usually present together? What features don't usually appear together?

In [ ]:
%%capture --no-display
def co_occurrence(n):
    top_f = top_features(n).index
    corr = gdf[top_f].notna().corr()
    # drop columns and rows related to geometry
    corr = corr.dropna(axis=1, how="all").dropna(axis=0, how="all")
    # sort the labels alphabetically for better clarity
    corr = corr.sort_index().reindex(sorted(corr.columns), axis=1)
    return corr

n = 20
corr = co_occurrence(n)
ax = sns.heatmap(corr, cmap="RdBu", center=0)
_ = ax.set_title("Correlation in occurrence of top 20 OSM features")


Key observations:
- The correlation in occurrence is generally positive among all features. 
- There are these hierarchical features (e.g. `brand`, `brand:wikidata`, `brand:wikipedia`) that demonstrate really high correlation.
- Some features appear to be negatively correlated, suggesting conflicting / exclusiveness of some features. For example, `building` and `amenity`.

#### Visualising the features

Q: How does the features look like on a map? How diverse / dense are they? How much data is there?

In [ ]:
# 9 features of inspect. Manually chose "interesting" features 
# out of the 25 most prominent ones listed in the bar plot above.

features = [
    "building", "geometry_area", "leisure", "natural", "amenity", 
    "brand", "shop", "operator", "building:levels", 
]

In [ ]:
def visualise(bbox, feature, legend, ax):
    gdf = clean_data(bbox)
    graph = osmnx.graph_from_bbox(*bbox)
    nodes, edges = osmnx.graph_to_gdfs(graph)
    edges.plot(ax=ax, linewidth=1, edgecolor="dimgray")
    
    is_num = pd.api.types.is_numeric_dtype(gdf[feature].dtype)
    if is_num:
        gdf[feature] = gdf[feature].astype(float)

    gdf.plot(
        ax=ax, 
        column=feature, 
        categorical=not is_num, 
        cmap="rainbow",
        alpha=0.9, 
        markersize=10, 
        edgecolor="b", 
        linewidth=0.5,
        legend=legend,
        missing_kwds={
            "color": "silver",
            "alpha": 0.4,
            "markersize": 10,
        }
    )

    ax.set_xlim(bbox[3], bbox[2])
    ax.set_ylim(bbox[1], bbox[0])
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title(feature)

def bbox_features(bbox):
    fig = plt.figure(figsize=(10, 10))

    for i, feature in enumerate(features):
        ax = fig.add_subplot(3, 3, i + 1)
        visualise(bbox, feature, False, ax)

    fig.tight_layout()

In [ ]:
bbox_features(bbox1)

In [ ]:
%%capture --no-display
bbox_features(bbox2)

Key observations
- Features have roughly a 50 / 50 split between "ways" (areas) and nodes. 
- Most features are indeed missing for most ways / nodes. 
- The selected tags don't have any features for roads.

## Question 3. Addressing a Property Price Prediction Question

For your final tick, we will be asking you to make house price predictions for a given location, date and property type in the UK. You will provide a function that takes input a latitude and longitude as well as the `property_type` (either type" of property (either `F` - flat, `S` - semidetached, `D` - detached, `T` - terraced or `O` other). Create this function in the `address.py` file, for example in the form,

```
def predict_price(latitude, longitude, date, property_type):
    """Price prediction for UK housing."""
    pass
```

We suggest that you use the following approach when building your prediction.

1. Select a bounding box around the housing location in latitude and longitude.
2. Select a data range around the prediction date.
3. Use the data ecosystem you have build above to build a training set from the relevant time period and location in the UK. Include appropriate features from OSM to improve the prediction.
4. Train a linear model on the data set you have created.
5. Validate the quality of the model.
6. Provide a prediction of the price from the model, warning appropriately if your validation indicates the quality of the model is poor.

The knowledge you need to do a first pass through this question will have been taught by end of lab session four (25th November 2021). You will likely want to review your answer as part of _refactoring_ your code shortly before hand in.


In [ ]:
from datetime import datetime

lat, long = config.config["sample_coords"]["christs"]
train_x, train_y, test_x, test_y, pred_x = address.train_test_set(
    lat, 
    long, 
    datetime(2016, 12, 1), 
    "F", 
    t_days=365 * 3, 
    bbox_length=2000,
    buffer_size=200,
)

In [ ]:
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

model = sm.GLM(train_y, train_x, family=sm.families.Poisson())
results = model.fit()

def plot(x, y, ax):
    y_pred = results.get_prediction(x).summary_frame(alpha=0.05)
    y_pred_mean = y_pred["mean"]
    y_pred_low = y_pred_mean - y_pred["mean_ci_lower"]
    y_pred_hi = y_pred["mean_ci_upper"] - y_pred_mean
    errs = np.concatenate([[y_pred_low], [y_pred_hi]])
    ax.scatter(y, y_pred_mean, s=5)

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

ax.set_yscale("log")
ax.set_xscale("log")
plot(train_x, train_y, ax)
plot(test_x, test_y, ax)

fig.tight_layout()

pred_y = results.get_prediction(pred_x).summary_frame(alpha=0.05)
pred_y["mean"]